##### 7.2.1 1차 미분 마스크
영상에서 밝기 검출 방법 : 밝기에 대한 기울기를 계산 -> 기울기의 크기 == 엣지

1차 미분 마스크 : 화소간 차분을 계산하도록 마스크 원소를 구성 -> 마스크 계수의 합은 0이 되어야 함

- 로버츠, 프리윗, 소벨 등의 1차 미분 마스크가 있음

로버츠 마스크 : 계산이 단순, 차분의 크기가 작음, 경계가 확실한 엣지만 추출하며, 잡음에 민감

In [ ]:
# 7.2.3 로버트 엣지 검출
import numpy as np, cv2
from Common.filters import filter

def differental(image, data1, data2):
    mask1 = np.array(data1, np.float32).reshape(3, 3)
    mask2 = np.array(data1, np.float32).reshape(3, 3)

    dst1 = filter(image, mask1)
    dst2 = filter(image, mask2)
    dst1, dst2 = np.abs(dst1), np.abs(dst2) # 절대값 계산 == 양수로 변경
    dst = cv2.magnitude(dst1, dst2) # 두 행렬의 크기 계산 == 엣지 강도 계산

    dst = np.clip(dst, 0, 255).astype('uint8')
    dst1 = np.clip(dst1, 0, 255).astype('uint8')
    dst2 = np.clip(dst2, 0, 255).astype('uint8')
    return dst, dst1, dst2


image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data1 = [-1, 0, 0,
         0, 1, 0,
         0, 0, 0]

data2 = [0, 0, -1,
         0, 1, 0,
         0, 0, 0]

mask1 = np.array(data1, np.float32).reshape(3,3)
mask2 = np.array(data2, np.float32).reshape(3,3)

dst, dst1, dst2 = differental(image, data1, data2)

cv2.imshow("image", image)
cv2.imshow("roberts edge", dst)
cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)
cv2.waitKey(0)


프리윗 마스트 : 로버츠의 단점을 보완하기 위해 고안, 원소의 배치가 수직 방향으로 구성, 에지의 방향도 수직

3번의 차분을 합하여 결과를 생성하기 때문에 엣지의 강도가 강하며, 수직과 수평 엣지를 동등하게 찾는데 효과적

In [ ]:
# 7.2.4 프리윗 엣지 검출
import numpy as np, cv2
from Common.filters import filter

def differental(image, data1, data2):
    mask1 = np.array(data1, np.float32).reshape(3, 3)
    mask2 = np.array(data1, np.float32).reshape(3, 3)

    dst1 = filter(image, mask1)
    dst2 = filter(image, mask2)
    dst = cv2.magnitude(dst1, dst2) # 두 행렬의 크기 계산 == 엣지 강도 계산

    dst = cv2.convertScaleAbs(dst) # 절대값 및 형변환
    dst1 = cv2.convertScaleAbs(dst1) # 절대값 및 형변환
    dst2 = cv2.convertScaleAbs(dst2) # 절대값 및 형변환
    return dst, dst1, dst2


image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data1 = [-1, 0, 1,
         -1, 0, 1,
         -1, 0, 1]

data2 = [-1, -1, -1,
         0, 0, 0,
         1, 1, 1]

dst, dst1, dst2 = differental(image, data1, data2)

cv2.imshow("image", image)
cv2.imshow("prewitt edge", dst)
cv2.imshow("dst1 : vertical mask", dst1)
cv2.imshow("dst2 : horizontal mask", dst2)
cv2.waitKey(0)


소벨 마스크 : 프리윗 마스크에서 중심 계수의 차분 비중이 2배

In [ ]:
# 7.2.5 소벨 엣지 검출
import numpy as np, cv2
from Common.filters import differental

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data1 = [-1, 0, 1,
         -2, 0, 2,
         -1, 0, 1]

data2 = [-1, -2, -1,
         0, 0, 0,
         1, 2, 1]

dst, dst1, dst2 = differental(image, data1, data2)

dst3 = cv2.Sobel(np.float32(image), cv2.CV_32F, 1,0,3) # x 방향 미분 - 수직 마스크
dst4 = cv2.Sobel(np.float32(image), cv2.CV_32F, 0,1,3) # y 방향 미분 - 수평 마스크

dst3 = cv2.convertScaleAbs(dst3) # 절대값 및 형변환
dst4 = cv2.convertScaleAbs(dst4) # 절대값 및 형변환


cv2.imshow("image", image)
cv2.imshow("prewitt edge", dst)
cv2.imshow("dst1 : vertical mask", dst1)
cv2.imshow("dst2 : horizontal mask", dst2)
cv2.imshow("dst3 : vertical CV", dst3)
cv2.imshow("dst4 : horizontal CV", dst4)
cv2.waitKey(0)


##### 7.2.2 2차 미분 마스크
1차 미분에서 한 번 더 미분 하는 방법

변화하는 영역의 중심에 위치한 엣지만을 검출

밝기가 점진적으로 변화되는 영역에 대해서는 반응을 보이지 않음

- 라플라시안, LoG, DoG 등이 있음

라플라시안 엣지 검출 : 중심 계수를 주변 계수의 4배로 하고, 부호는 반대로 하는 두 개의 4방향 마스크 사용

In [3]:
# 7.2.6 라플라시안 엣지 검출
import numpy as np, cv2

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

data1 = [[0, 1, 0],
         [1, -4, 1],
         [0, 1, 0]]

data2 = [[0, -1, 0],
         [-1, 4, -1],
         [0, -1, 0]]

mask4 = np.array(data1, np.int16)
mask8 = np.array(data2, np.int16)

dst1 = cv2.filter2D(image, cv2.CV_16S, mask4) # 회선 함수
dst2 = cv2.filter2D(image, cv2.CV_16S, mask8)
dst3 = cv2.Laplacian(image, cv2.CV_16S, 1) # 라플라시안 함수

dst1 = cv2.convertScaleAbs(dst1) # 절대값 및 형변환
dst2 = cv2.convertScaleAbs(dst2) # 절대값 및 형변환
dst3 = cv2.convertScaleAbs(dst3) # 절대값 및 형변환


cv2.imshow("image", image)
cv2.imshow("4-direction", dst1)
cv2.imshow("8-direction", dst2)
cv2.imshow("Laplacian", dst3)
cv2.waitKey(0)



NameError: name 'differental' is not defined

LoG, DoG : 잡음을 제거하는 마스크 (LoG를 보완하여 DoG를 만듦)

라플라시안은 잡음에 민감 -> 잡음 제거 후 라플라시안 수행 시 더욱 좋은 엣지가 검출됨



In [4]:
# 7.2.6 라플라시안 엣지 검출
import numpy as np, cv2

image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

gaus = cv2.GaussianBlur(image, (7, 7), 0, 0) # 가우시안 마스크 적용
dst1 = cv2.Laplacian(gaus, cv2.CV_16S, 7) # 라플라시안 수행

gaus1 = cv2.GaussianBlur(image, (3, 3), 0)
gaus2 = cv2.GaussianBlur(gaus1, (9, 9), 0)
dst2 = gaus1 - gaus2 # DoG

cv2.imshow("image", image)
cv2.imshow("LoG", dst1.astype('uint8'))
cv2.imshow("DoG", dst2)
cv2.waitKey(0)



-1

캐니 엣지 검출

검출 단계
1. 블러링을 통한 노이즈 제거
2. 화소 기울기의 강도와 방향 검출
3. 비최대치 억제 (이웃하는 화소보다 크면 유지, 아니면 제거)
4. 이력 임계값으로 에지 결정

In [2]:
# 7.2.8 캐니 엣지 검출
import numpy as np, cv2

# 비최대치 억제 함수
def nonmax_supperssion(sobel, direct):
    rows, cols = sobel.shape[:2]
    dst = np.zeros((rows, cols), np.float32)
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            values = sobel[i-1:i+2, j-1:j+2].flatten() # 중심 엣지 주변 9개 화소 가져옴
            first = [3, 0, 1, 2] # 이웃화소 좌표 4개
            id = first[direct[i, j]] # 현재 방향의 첫 번째 이웃 화소
            v1, v2 = values[id], values[8-id] # 두 이웃 화소

            dst[i, j] = sobel[i, j] if (v1 < sobel[i, j] > v2) else 0 # 비최대치 억제
    return dst

# 엣지 추적 함수
def trace(max_sobel, i, j, low):
    h, w = max_sobel.shape
    if (0 <= i < h and 0 <= j < w) == False: return # 추적 화소 범위 확인
    if pos_ck[i, j] > 0 and max_sobel[i, j] > low:
        ############# 여기 아래가 수행이 안됨
        print('hello')
        pos_ck[i, j] = 255 # 추적 좌표 완료 표시
        canny[i, j] = 255 # 엣지 지정

        # 재귀 호출
        trace(max_sobel, i-1, j-1, low)
        trace(max_sobel, i, j-1, low)
        trace(max_sobel, i+1, j-1, low)
        trace(max_sobel, i-1, j, low)
        trace(max_sobel, i, j, low)
        trace(max_sobel, i+1, j, low)
        trace(max_sobel, i-1, j+1, low)
        trace(max_sobel, i, j+1, low)
        trace(max_sobel, i+1, j+1, low)


# 이력 임계 처리 수행 함수
def hysteresis_th(max_sobel, low, high):
    rows, cols = max_sobel.shape[:2]
    for i in range(1, rows-1):
        for j in range(1, cols-1):
            if max_sobel[i, j] >= high: trace(max_sobel, i, j, low) # 높은 임계값 이상 시 추적


image = cv2.imread("images_07/test.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상 파일 읽기 오류")

pos_ck = np.zeros(image.shape[:2], np.uint8) # 추적 완료 점검 행렬
canny = np.zeros(image.shape[:2], np.uint8) # 캐니 엣지 행렬

# 캐니 엣지 검출
gaus_img = cv2.GaussianBlur(image, (5, 5), 0.3)
Gx = cv2.Sobel(np.float32(gaus_img), cv2.CV_32F, 1, 0, 3) # x방향 마스크
Gy = cv2.Sobel(np.float32(gaus_img), cv2.CV_32F, 0, 1, 3) # y방향 마스크
sobel = cv2.magnitude(Gx, Gy) 

directs = cv2.phase(Gx, Gy) / (np.pi/4) # 엣지 기울기 계산 및 근사
directs = directs.astype(int) % 4
max_sobel = nonmax_supperssion(sobel, directs)
hysteresis_th(max_sobel, 100, 150) # 이력 임계 값

canny2 = cv2.Canny(image, 100, 150)

cv2.imshow("image", image)
cv2.imshow("Canny", canny)
cv2.imshow("CV Canny", canny2)
cv2.waitKey(0)


-1